In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.


In [2]:
#Loading the training and testing data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [3]:
#Seed for reproducibilty
np.random.seed(1338)

#Selecting 6000 random examples from the test data
test_rows = np.random.randint(0,X_test.shape[0],6000)
X_test = X_test[test_rows]
Y = y_test[test_rows]
#Converting the output to binary classification(Six=1,Not Six=0)
Y_test = Y == 6
Y_test = Y_test.astype(int)

#Selecting the 5918 examples where the output is 6
X_six = X_train[y_train == 6]
Y_six = y_train[y_train == 6]
#Selecting the examples where the output is not 6
X_not_six = X_train[y_train != 6]
Y_not_six = y_train[y_train != 6]

#Selecting 6000 random examples from the data that contains only the data where the output is not 6
random_rows = np.random.randint(0,X_six.shape[0],6000)
X_not_six = X_not_six[random_rows]
Y_not_six = Y_not_six[random_rows]

In [4]:
#Appending the data with output as 6 and data with output as not six
X_train = np.append(X_six,X_not_six)
#Reshaping the appended data to appropraite form
X_train = X_train.reshape(X_six.shape[0] + X_not_six.shape[0], 784)
#Appending the labels and converting the labels to binary classification(Six=1,Not Six=0)
Y_labels = np.append(Y_six,Y_not_six)
Y_train = Y_labels == 6 
Y_train = Y_train.astype(int)

In [5]:
#Converting the classes to its binary categorical form
nb_classes = 2
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [6]:
#Initializing the values for the multi layer perceptron
batch_size = 128
nb_epoch = 20
#Activation Functions
activations = ['relu','tanh','sigmoid']

In [7]:
#Function for constructing the multi layer perceptron
def build_model(activation):
    
    np.random.seed(1338)
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation(activation))
    model.add(Dropout(0.2))
    model.add(Dense(512))
    model.add(Activation(activation))
    model.add(Dropout(0.2))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1,
              validation_data=(X_test, Y_test))
          

    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [8]:
for i in activations:
    print('ACTIVATION',i,'\n')
    %timeit -n1 -r1 build_model(i)
    print('\n')

ACTIVATION relu 

Train on 11918 samples, validate on 6000 samples
Epoch 1/20
11918/11918 [==============================] - 4s - loss: 0.1219 - acc: 0.9537 - val_loss: 0.0598 - val_acc: 0.9760
Epoch 2/20
11918/11918 [==============================] - 4s - loss: 0.0389 - acc: 0.9867 - val_loss: 0.0251 - val_acc: 0.9928
Epoch 3/20
11918/11918 [==============================] - 4s - loss: 0.0209 - acc: 0.9930 - val_loss: 0.0248 - val_acc: 0.9915
Epoch 4/20
11918/11918 [==============================] - 4s - loss: 0.0135 - acc: 0.9961 - val_loss: 0.0266 - val_acc: 0.9912
Epoch 5/20
11918/11918 [==============================] - 4s - loss: 0.0091 - acc: 0.9971 - val_loss: 0.0324 - val_acc: 0.9898
Epoch 6/20
11918/11918 [==============================] - 4s - loss: 0.0059 - acc: 0.9982 - val_loss: 0.0500 - val_acc: 0.9853
Epoch 7/20
11918/11918 [==============================] - 5s - loss: 0.0035 - acc: 0.9988 - val_loss: 0.0474 - val_acc: 0.9878
Epoch 8/20
11918/11918 [====================

In [9]:
#%timeit -n1 -r1 build_model()